In [19]:
from langgraph.graph import StateGraph,START,END
from typing import TypedDict
from langchain_groq import ChatGroq

from dotenv import load_dotenv

In [20]:
load_dotenv()

model = ChatGroq(model="llama-3.1-8b-instant")

In [21]:
class Batsman(TypedDict):
    runs: int 
    balls: int
    fours: int
    sixes: int 
    
    sr: float
    bpb: float
    boundry_percentage: float

In [22]:
def calculate_sr(state:Batsman):
    sr=state['runs']/state['balls']*100
    #state['sr'] = sr
    
    return {'sr':sr}

In [23]:
def calculate_bpb(state:Batsman):
    bpb=state['balls']/(state['fours']+state['sixes'])
    
    #state['bpb'] = bpb
    
    return {'bpb':bpb}

In [24]:
def calculate_boundry_percentage(state:Batsman):
    boundry_percentage=(((state['fours']*4) +(state['sixes']*6))/state['balls'])*100
    
    #state['boundry_percentage'] = boundry_percentage
    
    return {'boundry_percentage':boundry_percentage}

In [25]:
def summary(state:Batsman):
    
    summary =  f"""
    strike_rate: {state['sr']},
    balls_per_boundry: {state['bpb']},
    boundry_percentage: {state['boundry_percentage']}
    """
    state['summary'] = summary
    return state

In [26]:
graph = StateGraph(Batsman)

graph.add_node('calculate_sr',calculate_sr)
graph.add_node('calculate_bpb',calculate_bpb)
graph.add_node('calculate_boundry_percentage',calculate_boundry_percentage)
graph.add_node('summary',summary)


graph.add_edge(START,'calculate_sr')
graph.add_edge(START,'calculate_bpb')
graph.add_edge(START,'calculate_boundry_percentage')

graph.add_edge('calculate_sr','summary')
graph.add_edge('calculate_bpb','summary')
graph.add_edge('calculate_boundry_percentage','summary')

graph.add_edge('summary',END)

workflow = graph.compile()



In [28]:
input_state = {
    'runs': 120,
    'balls': 100,
    'fours': 10,
    'sixes': 5
}

workflow.invoke(input_state)




{'runs': 120,
 'balls': 100,
 'fours': 10,
 'sixes': 5,
 'sr': 120.0,
 'bpb': 6.666666666666667,
 'boundry_percentage': 70.0}